You need to download the table from the moodle and this file will filter out the late hour penalties for all of them automatically

In [269]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [270]:
entry_to_name = dict()

In [271]:
def parseHours(lateData):
    lateData = lateData.lower()
    if ("submitted" in lateData) or ("not" in lateData):
        return 0
    lateData = lateData.split(' ')
    totalDay: float = 0
    
    if lateData[1] == 'days' or lateData[1] == 'day':
        totalDay += int(lateData[0]) * 24
        if len(lateData) > 3:
            totalDay += int(lateData[2])
    if lateData[1] == 'hours' or lateData[1] == 'hour':
        totalDay += int(lateData[0])
        if len(lateData) > 3:
            totalDay += float(int(lateData[2]) / 60)
    if lateData[1] == 'mins' or lateData[1] == 'min':
        totalDay += float(int(lateData[0]) / 60)
    return totalDay

In [272]:
ids = [f'a{i}' for i in range(0, 5)]
for id in ids:
    with open(os.path.join(f'{id}_table.html')) as f:
        soup = BeautifulSoup(f, 'html.parser')

    rows = soup.find("table").find("tbody").find_all("tr")

    late_df = {"Name": [], "EntryNo": [], f"LateHours_{id}": []}
    for row in rows:
        cells = row.find_all("td")
        store = list()
        for cell in cells[2:7]:
            store.append(cell.get_text())
        store = [val.strip() for val in store if len(val.strip()) > 0]
        if len(store):
            lateHours = store[2].split('\n')[-1].strip()
            if not('late' in lateHours):
                lateHours = '0 day 0 hours'
            lateHours = lateHours.replace('late', '')
            late_df["Name"].append(store[0])
            late_df["EntryNo"].append(store[1])
            entry_to_name[store[1]] = store[0]
            late_df[f"LateHours_{id}"].append(lateHours)

    late_df = pd.DataFrame(late_df)
    late_df.set_index("EntryNo", inplace=True)
    late_df.to_csv(os.path.join(f'late_{id}.csv'))

In [273]:
final_df = pd.read_csv('late_a0.csv')
for i in range(1, 5):
    file_path = os.path.join(f'late_a{i}.csv')
    late_df = pd.read_csv(file_path)
    late_df.set_index('EntryNo', inplace=True)
    final_df = final_df.merge(late_df[[f'LateHours_a{i}']], left_on='EntryNo', right_on='EntryNo', how='outer')

final_df.head()
final_df = final_df.replace(np.nan, 'Not Submitted')
for _, row in final_df.iterrows():
    name = entry_to_name[row['EntryNo']]
    row['Name'] = name 

final_df.set_index('EntryNo', inplace=True)
total_late_hours = dict()
for idx, row in final_df.iterrows():
    sum = 0
    for col in [f'LateHours_a{i}' for i in range(0, 5)]:
        sum += parseHours(row[col])
    sum = round(sum, 2)
    total_late_hours[idx] = sum

final_df['TotalLateHours'] = list(total_late_hours.values())
consider_for_penalty = [max(0, val - 120) for val in list(total_late_hours.values())]
final_df['ConsiderForPenalty'] = consider_for_penalty
final_df['PenaltyMarks'] = [float(val / 12) * 0.5 for val in consider_for_penalty]
final_df.to_csv('final.csv')